# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

Setup an LDA model and discretize using
a single kpoint and a small `Ecut` of 5 Hartree.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model, 5, kgrid=(1, 1, 1))

PlaneWaveBasis (Ecut=5.0, 1 kpoints)

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.116106e+02     1.534717e+00
 * time: 0.08383584022521973
     1     1.063641e+01     1.060039e+00
 * time: 0.9164028167724609
     2    -1.186287e+01     1.174268e+00
 * time: 1.0217528343200684
     3    -3.380976e+01     8.916643e-01
 * time: 1.1747570037841797
     4    -4.716865e+01     6.069002e-01
 * time: 1.3165099620819092
     5    -5.692000e+01     2.035238e-01
 * time: 1.4556019306182861
     6    -5.977594e+01     1.227533e-01
 * time: 1.5655179023742676
     7    -6.085862e+01     5.125252e-02
 * time: 1.6847758293151855
     8    -6.129961e+01     5.181462e-02
 * time: 1.7873058319091797
     9    -6.154474e+01     3.131817e-02
 * time: 1.8975839614868164
    10    -6.178196e+01     2.745777e-02
 * time: 2.014312982559204
    11    -6.196131e+01     2.239862e-02
 * time: 2.116529941558838
    12    -6.204712e+01     1.999791e-02
 * time: 2.227695941925049
    13    -6.211978e+01     1.528497e-02
 * time: 2.345435857772

In [5]:
scfres.energies

Energy breakdown:
    Kinetic             25.7671070
    AtomicLocal         -18.8557646
    AtomicNonlocal      14.8522614
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485368 
    Xc                  -19.3336820

    total               -62.261666460356
